<a href="https://colab.research.google.com/github/hikaruyaku/KIKAGAKU-/blob/master/%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E5%AE%9F%E8%A3%85%EF%BC%88%E5%9B%9E%E5%B8%B0%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ライブラリのインストール
!pip install pytorch_lightning

In [2]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving housing.csv to housing (3).csv


In [0]:
# データの読込
df = pd.read_csv("housing.csv")

In [4]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [5]:
df.shape

(506, 14)

In [6]:
t = df['y']
x = df.drop('y', axis=1)
x.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [7]:
x.shape, t.shape

((506, 13), (506,))

In [8]:
type(x), type(t)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [0]:
import torch

# PyTorch で学習に使用できる形式へ変換
x = torch.tensor(x.values, dtype=torch.float32)
t = torch.tensor(t.values, dtype=torch.float32)

In [0]:
import torch.utils.data

In [11]:
# 入力変数と目的変数をまとめて、一つのオブジェクト dataset に変換
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [12]:
# (入力変数、目的変数) のようにタプルで格納されている
dataset[0]

(tensor([6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01, 6.5750e+00,
         6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01, 3.9690e+02,
         4.9800e+00]), tensor(24.))

In [13]:
# 各データセットのサンプル数を決定
# train : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val   = int(len(dataset) * 0.2)
n_test  = len(dataset) - n_train - n_val

# サンプル数の確認
n_train, n_val, n_test

(303, 101, 102)

In [0]:
# ランダムに分割を行う為、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

In [16]:
# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [0]:
# 学習データに対する処理
class TrainNet(pl.LightningModule):
    
    @pl.data_loader 
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True)
    
    def training_step(self, batch, batch_nb):
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

In [0]:
# 検証データに対する処理
class ValidationNet(pl.LightningModule):

    @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)

    def validation_step(self, batch, batch_nb):
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'val_loss': loss}
        return results

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        results = {'val_loss': avg_loss}
        return results

In [0]:
# テストデータに対する処理
class TestNet(pl.LightningModule):

    @pl.data_loader
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)

    def test_step(self, batch, batch_nb):
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'test_loss': loss}
        return results

    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        results = {'test_loss': avg_loss}
        return results

In [0]:
# 学習データ、検証データ、テストデータへの処理を継承したクラス
class Net(TrainNet, ValidationNet, TestNet):
    
    def __init__(self, input_size=13, hidden_size=5, output_size=1, batch_size=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
 #       self.bn = nn.BatchNorm1d(input_size)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
#        x = self.bn(x)
        x = self.fc2(x)
        return x
    
    # New: 平均ニ乗誤差
    def lossfun(self, y, t):
        return F.mse_loss(y, t)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

In [35]:
from pytorch_lightning import Trainer
# 再現性を確保
torch.manual_seed(0)

# インスタンス化
net = Net.to(device)

trainer = Trainer(max_epochs=100, gpus=[0])

# 学習の実行
trainer.fit(net)

AttributeError: ignored

In [30]:
trainer.test()

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You passed  in a `test_dataloader` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(nan, device='cuda:0')}
--------------------------------------------------------------------------------



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


In [31]:
trainer.callback_metrics

{'epoch': 99,
 'loss': tensor(nan, device='cuda:0'),
 'test_loss': tensor(nan, device='cuda:0'),
 'val_loss': tensor(nan, device='cuda:0')}

In [32]:
#元のスケールに調整
torch.sqrt(torch.tensor(trainer.callback_metrics["test_loss"]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


tensor(nan, device='cuda:0')